# SAC vs ObsPy — Equivalent Processing Workflow (ObsPy Notebook)

This notebook mirrors a typical SAC macro:

1. Read waveform files (SAC or MiniSEED)
2. Demean, detrend, taper
3. Bandpass filter (zero-phase)
4. (Optional) decimate / resample
5. Plot before/after
6. Write outputs back to SAC (or other formats)

**Edit the configuration cell below** (glob, filter band, output mode).


In [ ]:
# --- Configuration (edit me) ---
from pathlib import Path

INPUT_GLOB = "*.SAC"      # e.g., "*.SAC" or "*.mseed"
OUT_DIR = Path("processed")
OVERWRITE = False         # True to overwrite original files (be careful!)
SUFFIX = ".bp_1_10"       # appended to filename stem if not overwriting

# Bandpass settings (Hz)
FREQMIN = 1.0
FREQMAX = 10.0
CORNERS = 4
ZEROPHASE = True          # SAC bp ... p 2 ~ zerophase

# Preprocessing
TAPER_MAX_PERCENTAGE = 0.05

# Optional decimation (set to None to disable)
DECIMATE_FACTOR = None    # e.g., 2, 4, 5

OUT_DIR


In [ ]:
# --- Imports ---
from obspy import read
import matplotlib.pyplot as plt

# (Optional) for nicer plots of many traces, you can use st.plot()


In [ ]:
# --- Discover files ---
files = sorted(Path(".").glob(INPUT_GLOB))
if not files:
    raise FileNotFoundError(f"No files match {INPUT_GLOB!r} in {Path('.').resolve()}")

print(f"Found {len(files)} file(s). Showing first 10:")
for fn in files[:10]:
    print(" ", fn)


## Preview one file (raw)

In [ ]:
# Pick a file to preview
preview_fn = files[0]
st_raw = read(str(preview_fn))
st_raw.plot()  # interactive plot


## Define the processing function (ObsPy equivalent of the SAC macro)

In [ ]:
def process_stream(st):
    """In-place processing to mirror the SAC macro steps."""
    # Demean + detrend
    st.detrend("demean")
    st.detrend("linear")

    # Taper
    st.taper(max_percentage=TAPER_MAX_PERCENTAGE)

    # Bandpass (SAC: bp c f1 f2 n 4 p 2)
    st.filter("bandpass",
              freqmin=FREQMIN, freqmax=FREQMAX,
              corners=CORNERS, zerophase=ZEROPHASE)

    # Optional decimation (with anti-aliasing filter)
    if DECIMATE_FACTOR is not None:
        st.decimate(factor=int(DECIMATE_FACTOR), no_filter=False)

    return st


## Batch process and write outputs

In [ ]:
OUT_DIR.mkdir(exist_ok=True)

for fn in files:
    st = read(str(fn))
    st_proc = process_stream(st)

    # Decide output path
    if OVERWRITE:
        out_fn = fn
    else:
        out_fn = OUT_DIR / (fn.stem + SUFFIX + fn.suffix)

    # Write as SAC if the input is SAC; otherwise you can choose another format
    # ObsPy will infer format from extension poorly; so specify format explicitly for SAC:
    if fn.suffix.lower() == ".sac":
        st_proc.write(str(out_fn), format="SAC")
    else:
        # For MiniSEED example:
        st_proc.write(str(out_fn), format="MSEED")

    print(f"Wrote: {out_fn}")


## Compare before/after on one example

In [ ]:
example_in = files[0]
st0 = read(str(example_in))

if OVERWRITE:
    example_out = example_in
else:
    example_out = OUT_DIR / (example_in.stem + SUFFIX + example_in.suffix)

st1 = read(str(example_out))

# Plot raw then processed
st0.plot()
st1.plot()


## Notes / Common tweaks

- To mimic SAC's `w over`, set `OVERWRITE = True`.
- If you want *new* SAC files but keep them alongside inputs, set `OUT_DIR = Path('.')`.
- For response removal, use `read_inventory()` + `st.remove_response(...)` before filtering.
- For multi-station / multi-channel workflows, you can add `st.select(...)` logic.
